In [1]:
import numpy as np
from rdkit.Chem import AllChem
import polars as pl
import xgboost as xgb
import skopt
import sklearn

import math
from typing import Dict, Tuple, List, Optional

In [2]:
# load dataset
# load uniprot to seq map
# select a single id from uniprot map
# filter dataset to compounds for that kinase
# split data (80/20) for test validate
# run through 50, 100, 200? opt loops for xgboost
# features?
# save finished model to some sort of directory

In [3]:
def index_kinases(tsv_path: str, idx: int) -> Optional[str]:
    ''' read Uniprot TSV file and get the Uniprot ID at line `idx` '''
    with open(tsv_path, 'rt') as f:
        for i, line in enumerate(f):
            if i == idx: return line.split('\t')[0]
    
    return None
    
index_kinases('data/map-uniprot-seq.tsv', 1131)

'P11799'

In [32]:
pl.scan_parquet('data/full-median.parquet').columns

['compound_chembl_id',
 'canonical_smiles',
 'standard_inchi',
 'uniprot',
 'standard_type',
 'Median Activity [-logP]']

In [18]:
def ecfp_from_inchi(inchi, R=2):
    mol = AllChem.MolFromInchi(inchi)
    return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, R), dtype=np.uint8)


raw = (pl.scan_parquet('data/full-median.parquet')
    .select('compound_chembl_id', 'canonical_smiles', 'standard_inchi')
    .unique()
)
# raw = (pl.scan_parquet('data/full-median.parquet')
#      .select('compound_chembl_id', 'canonical_smiles', 'standard_inchi')
#      .unique()
#      .with_columns([
#          pl.col('standard_inchi').apply(ecfp_from_inchi, return_dtype=pl.List(pl.UInt8)).alias('ECFP')
#      ])
#      .select('compound_chembl_id', 'ECFP')
#      .fetch(20)
# )

# test = pl.scan_parquet('data/full-median.parquet').fetch(1)[0, 'standard_inchi']

In [19]:
# 128584
# 253238
df = raw.collect()
df

compound_chembl_id,canonical_smiles,standard_inchi
str,str,str
"""CHEMBL3680427""","""C[C@@H](Nc1ncn…","""InChI=1S/C25H1…"
"""CHEMBL3894803""","""Cc1cnc(Nc2cnn(…","""InChI=1S/C21H2…"
"""CHEMBL3970953""","""Cc1cncc(-c2cc3…","""InChI=1S/C19H1…"
"""CHEMBL3701277""","""COc1cc(C#Cc2cn…","""InChI=1S/C27H3…"
"""CHEMBL248853""","""CCn1c(-c2ccnc(…","""InChI=1S/C13H1…"
"""CHEMBL3950222""","""O=C(Cc1nc(N2CC…","""InChI=1S/C20H2…"
"""CHEMBL574236""","""COc1ccccc1-n1c…","""InChI=1S/C22H1…"
"""CHEMBL4740635""","""O=c1c2cc(-c3cn…","""InChI=1S/C18H1…"
"""CHEMBL4457529""","""COCC(C)(C)C#Cc…","""InChI=1S/C20H2…"


In [20]:
it = zip(df['compound_chembl_id'], df['standard_inchi'])
output = {i: ecfp_from_inchi(key, 2) for i, key in it}

[13:56:29] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match)

[13:56:30] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Mobile-H groups: Missing, Attachment points; Charge(s): Do not match; Proton balance: Does not match)

[13:56:48] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Attachment points, Number)

[13:57:08] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Attachment points, Number)

[13:57:20] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match)

[13:57:21] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number; Mobile-H groups: Attachment points, Number)

[13:57:32] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Charge(s): Do not match)

[13:57:47] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: L

In [25]:
import pickle

with open('data/map-ecfp2.pkl', 'wb') as f:
    pickle.dump(output, f)

In [35]:
target = index_kinases('data/map-uniprot-seq.tsv', 2)
MEASURES = ['KD', 'KI', 'EC50']
ACTIVITY = 'Median Activity [-logP]'

data = (pl.scan_parquet('data/full-median.parquet')
 .filter(pl.col('uniprot') == target)
 .filter(pl.col('standard_type').is_in(MEASURES))
 .select(['compound_chembl_id', ACTIVITY])
 .groupby('compound_chembl_id').median()
).collect()


In [36]:
labels = data[ACTIVITY].to_numpy()

In [43]:
feats = np.array([output[chembl] for chembl in data['compound_chembl_id']])

In [44]:
dtrain = xgb.DMatrix(feats, label=labels)
bst = xgb.train({}, dtrain)


In [144]:
from skopt import BayesSearchCV
from skopt.space import space
from sklearn.model_selection import train_test_split

np.int = int

X_train, X_val, y_train, y_val = train_test_split(feats, labels, train_size=0.8, test_size=0.2, random_state=42)

xgbr = xgb.XGBRegressor(tree_method='hist')

opt = BayesSearchCV(
    xgbr,
    {
        'n_estimators': space.Integer(1, 500, dtype=np.int64),
        'max_depth': (1, 20),
        'max_leaves': (0, 20),
        'subsample': (0.5, 1.0, 'uniform'),
    },
    n_iter=200,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)

opt.fit(X_train, y_train)

/Users/mwc/Documents/cobb/cobb2066/projects/08/dev/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/mwc/Documents/cobb/cobb2066/projects/08/dev/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/mwc/Documents/cobb/cobb2066/projects/08/dev/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/mwc/Documents/cobb/cobb2066/projects/08/dev/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/mwc/Documents/cobb/cobb2066/projects/08/dev/lib/python3.1

BayesSearchCV(cv=5,
              estimator=XGBRegressor(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
              n_iter=200, n_jobs=-1, scoring='neg_root_mean_squared_error',
              search_spaces={'max_depth': (1, 20), 'max_leaves': (0, 20),
                             'n_estimators': Integer(low=1, high=500, prior='uniform', transform='normalize'),
                             'subsample': (0.5, 1.0, 'uniform')})

In [145]:
opt.best_params_, opt.best_score_

(OrderedDict([('max_depth', 1),
              ('max_leaves', 20),
              ('n_estimators', 35),
              ('subsample', 1.0)]),
 -0.7162245920770927)

In [146]:
pl.DataFrame(opt.cv_results_).sort('rank_test_score')

mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_leaves,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
f64,f64,f64,f64,object,object,object,object,struct[4],f64,f64,f64,f64,f64,f64,f64,i32
0.028181,0.002007,0.000986,0.000033,1,20,35,1.0,"{1,20,35,1.0}",-0.796864,-0.563692,-0.699417,-0.735372,-0.785777,-0.716225,0.083971,1
0.021851,0.000189,0.000973,0.000028,1,0,35,1.0,"{1,0,35,1.0}",-0.796864,-0.563692,-0.699417,-0.735372,-0.785777,-0.716225,0.083971,1
0.022616,0.001465,0.000957,0.000013,1,20,18,1.0,"{1,20,18,1.0}",-0.794987,-0.584606,-0.68834,-0.746775,-0.766843,-0.71631,0.074566,3
0.01717,0.000364,0.000983,0.000048,1,20,18,1.0,"{1,20,18,1.0}",-0.794987,-0.584606,-0.68834,-0.746775,-0.766843,-0.71631,0.074566,3
0.017534,0.000485,0.001006,0.000035,1,20,18,1.0,"{1,20,18,1.0}",-0.794987,-0.584606,-0.68834,-0.746775,-0.766843,-0.71631,0.074566,3
0.022282,0.000656,0.000964,0.000067,1,20,37,1.0,"{1,20,37,1.0}",-0.797028,-0.559447,-0.692908,-0.733651,-0.79937,-0.716481,0.088169,6
0.022376,0.000435,0.000991,0.000039,1,20,37,1.0,"{1,20,37,1.0}",-0.797028,-0.559447,-0.692908,-0.733651,-0.79937,-0.716481,0.088169,6
0.022279,0.000402,0.00097,0.000033,1,0,37,1.0,"{1,0,37,1.0}",-0.797028,-0.559447,-0.692908,-0.733651,-0.79937,-0.716481,0.088169,6
0.023352,0.000713,0.000968,0.000018,1,0,19,1.0,"{1,0,19,1.0}",-0.794977,-0.584052,-0.686836,-0.746809,-0.769757,-0.716486,0.075281,9


In [147]:
opt.score(X_val,y_val)

-0.718926558715587

In [148]:
np.corrcoef(opt.predict(X_val), y_val).min()

0.5554223351169976

In [149]:
math.sqrt(sklearn.metrics.mean_squared_error(y_val, opt.predict(X_val)))

0.718926558715587

In [179]:
sklearn.metrics.r2_score(y_val, opt.predict(X_val))

0.25017011573112047

In [181]:
from sklearn.model_selection import cross_val_score

test = xgb.XGBRegressor(n_jobs=-1, **opt.best_params_)
test.fit(feats, labels)
(
    -cross_val_score(test, feats, labels, scoring='neg_root_mean_squared_error').mean(),
    cross_val_score(test, feats, labels, scoring='r2').mean(),
    np.corrcoef(test.predict(feats), labels).min()
)


(0.6997270042350988, 0.1796536693179907, 0.7384592609936014)

In [182]:
test.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 1,
 'max_leaves': 20,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 35,
 'n_jobs': -1,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 1.0,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [184]:
pl.DataFrame({"a": 'hello', "b": 2})

a,b
str,i64
"""hello""",2
